# Base Case

In [1]:
# Feb 23
import math
import pdb

In [20]:
import pandas
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook as tqdm

# Load data + Preprocess

In [3]:
# main = pandas.read_csv('../data/processed/completed_data/mainset.csv', index_col=0)
# addi = pandas.read_csv('../data/processed/completed_data/addi.csv', index_col=0)

In [4]:
# data = pandas.concat([main.reset_index(), addi.reset_index()], axis=0, sort=False).reset_index(drop=True)

In [5]:
# data.rename(columns={'index':'tvdss'}, inplace=True)

In [5]:
# data['well'].unique()

In [7]:
# data.drop(columns=['md', 'DEPTH', 'tvd', 'TVD'], inplace=True) # 'md'

In [18]:
# data.columns

Index(['tvdss', 'BS', 'CALI', 'DENS', 'DRHO', 'DTC', 'GR', 'NEUT', 'PEF',
       'RESD', 'RESM', 'RESS', 'SP', 'fm', 'well', 'DTS', 'GR_CORR',
       'NEUT_CORR', 'RESD_CORR', 'RESS_CORR', 'TEMP', 'TENS'],
      dtype='object')

In [21]:
# data.to_csv('../data/processed/completed_data/datablob.csv')

In [3]:
df = pandas.read_csv('../data/datablob.csv', index_col=0)

In [4]:
df.tail().T

,247525,247526,247527,247528,247529
tvdss,1483.81,1483.95,1484.09,1484.24,1484.38
BS,8.5,8.5,8.5,8.5,8.5
CALI,NaN,NaN,NaN,NaN,NaN
DENS,NaN,NaN,NaN,NaN,NaN
DRHO,NaN,NaN,NaN,NaN,NaN
DTC,NaN,NaN,NaN,NaN,NaN
GR,NaN,NaN,NaN,NaN,NaN
NEUT,NaN,NaN,NaN,NaN,NaN
PEF,NaN,NaN,NaN,NaN,NaN
RESD,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df[pandas.notnull(df['fm'])]
df.head().T

,1717,1718,1719,1720,1721
tvdss,75.214,75.214,75.366,75.518,75.671
BS,NaN,8.5,8.5,8.5,8.5
CALI,NaN,9.767,9.905,9.958,9.951
DENS,NaN,2.137,2.104,2.136,2.117
DRHO,NaN,-0.007,-0.005,-0.003,0
DTC,NaN,131.311,131.458,131.971,131.583
GR,NaN,84.724,83.405,83.974,88.314
NEUT,NaN,0.429,0.408,0.388,0.421
PEF,NaN,2.262,2.396,2.399,2.278
RESD,NaN,78.705,75.329,80.198,83.477


In [ ]:
df.head()

In [6]:
df.columns

Index(['tvdss', 'BS', 'CALI', 'DENS', 'DRHO', 'DTC', 'GR', 'NEUT', 'PEF',
       'RESD', 'RESM', 'RESS', 'SP', 'fm', 'well', 'DTS', 'GR_CORR',
       'NEUT_CORR', 'RESD_CORR', 'RESS_CORR', 'TEMP', 'TENS'],
      dtype='object')

In [7]:
df['fm'].unique()

array(['Matemateaonga Fm', 'Kiore SST', 'Urenui Fm', 'Urenui B SST',
       'Urenui A SST', 'Egmont Volcanics', 'Tangahoe Fm', 'Mt Messenger',
       'MM3 SST', 'MM1 SST', 'Manganui Fm', 'Moki',
       'Lower Manganui Marker', 'Tikorangi LST', 'MM2 SST',
       'Lower Kiore SST', 'Top Urenui Fm', 'Basal Urenui',
       'Intra Kiore SST', 'Upper Manganui', 'Cheal SST'], dtype=object)

In [8]:
df['well'].unique()

array(['cheal-a10', 'cheal-a11', 'cheal-a12', 'cheal-b8', 'cheal-c3',
       'cheal-c4', 'cheal-g1', 'cheal-g2', 'cheal-g3', 'cheal-a6',
       'cheal-b1', 'cheal-1', 'cheal-2', 'cheal-a4', 'cheal-a3x',
       'cheal-b4', 'cheal-a7'], dtype=object)

In [9]:
from fastai.tabular import *
from fastai.metrics import fbeta

In [10]:
dep_var = 'fm'
cat_names = ['well']
cont_names = ['tvdss', 'BS', 'CALI', 'DENS', 'DRHO', 'DTC', 'GR', 'NEUT', 'PEF','RESD', 'RESM', 'RESS', 'SP', 'DTS', 'GR_CORR','NEUT_CORR', 'RESD_CORR', 'RESS_CORR', 'TEMP', 'TENS']
procs = [FillMissing, Categorify, Normalize]

In [11]:
test = TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names)

In [12]:
data = (TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .random_split_by_pct()
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [103]:
len(df)

242088

In [86]:
data.show_batch(rows=10)

well,BS_na,CALI_na,DENS_na,DRHO_na,DTC_na,GR_na,NEUT_na,PEF_na,RESD_na,RESM_na,RESS_na,SP_na,DTS_na,GR_CORR_na,NEUT_CORR_na,RESD_CORR_na,RESS_CORR_na,TEMP_na,TENS_na,tvdss,BS,CALI,DENS,DRHO,DTC,GR,NEUT,PEF,RESD,RESM,RESS,SP,DTS,GR_CORR,NEUT_CORR,RESD_CORR,RESS_CORR,TEMP,TENS,target
cheal-g2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.4354,-0.2849,-0.0489,0.1297,2.1194,-0.0437,0.5117,0.2619,0.9053,-0.0513,-0.0384,-0.0229,-0.2815,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,Urenui Fm
cheal-1,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,1.8082,-0.2849,0.3387,1.6320,-0.9135,-1.5436,1.3031,-2.3052,0.5201,-0.0460,-0.0149,-0.0155,0.7454,-0.0037,4.8469,-3.7662,-0.2395,0.1692,3.5602,0.3138,Moki
cheal-g1,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.0099,-0.2849,-0.1872,-0.2380,1.3384,0.7544,0.8408,0.2796,-0.1830,-0.0153,-0.0064,0.0006,0.7634,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,Matemateaonga Fm
cheal-a11,False,False,False,True,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,0.3606,-0.2849,-0.2027,0.3044,-0.1619,-0.1463,0.9209,-0.3569,0.8156,-0.0527,-0.0400,-0.0262,0.0928,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,Urenui B SST
cheal-c4,False,False,False,True,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,0.3294,-0.2849,-0.0997,-1.0839,-0.1619,0.2336,-0.7758,-0.3922,-2.0214,-0.0505,-0.0477,-0.0391,0.0928,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,Urenui B SST
cheal-b8,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,1.8300,-0.2849,-0.0313,1.6835,-1.3795,-1.2890,1.3893,-0.8872,0.7871,-0.0424,-0.0321,-0.0141,0.2785,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,MM1 SST
cheal-c3,False,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,-1.0622,1.7977,-0.1004,0.0194,-0.1619,-0.0542,-1.7521,0.0851,0.0330,-0.0222,-0.0149,-0.0190,0.0928,-0.0037,-0.0158,0.1551,-0.1412,-0.0908,-0.0380,-0.0453,Matemateaonga Fm
cheal-b1,False,False,False,False,False,False,False,True,False,True,False,False,True,True,False,True,True,False,False,0.6624,-0.2849,-0.2078,0.6906,-1.0268,-0.3270,0.8627,0.4887,0.0330,-0.0590,-0.0149,-0.0281,0.1498,-0.0037,-0.0158,1.5145,-0.1412,-0.0908,1.2077,1.0156,Mt Messenger
cheal-b4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,0.1344,-0.2849,-0.1029,-0.0808,-1.1791,0.1842,-0.2470,0.0504,-0.1590,-0.0366,-0.0289,-0.0233,-0.2201,-0.0037,-0.0158,0.8568,-0.1412,-0.0908,1.0974,1.4399,Kiore SST
cheal-b4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,1.2336,-0.2849,0.1421,1.4840,-1.2737,-1.1760,1.0239,-1.0024,0.8572,-0.0459,-0.0325,-0.0160,-0.2342,-0.0037,-0.0158,-0.9789,-0.1412,-0.0908,3.6012,4.3167,Manganui Fm


In [52]:
def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True)->Rank0Tensor:
    "Computes the f_beta between `preds` and `targets`"
#     pdb.set_trace()
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    return f1_score(y_true, y_pred, average='micro')

In [41]:
def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True)->Rank0Tensor:
    "Computes the f_beta between `preds` and `targets`"
#     pdb.set_trace()
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = pandas.np.argmax(y_pred, axis=1).float().cuda().unsqueeze(1)
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean()

In [13]:
learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])

In [14]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.289441,0.208737,0.926451


In [101]:
type(data)

fastai.tabular.data.TabularDataBunch

In [128]:
data.test_ds.x[0]

TabularLine well cheal-a10; BS_na True; CALI_na True; DENS_na True; DRHO_na True; DTC_na True; GR_na True; NEUT_na True; PEF_na True; RESD_na True; RESM_na True; RESS_na True; SP_na True; DTS_na True; GR_CORR_na True; NEUT_CORR_na True; RESD_CORR_na True; RESS_CORR_na True; TEMP_na True; TENS_na True; tvdss -1.2073; BS -0.2845; CALI -0.1005; DENS 0.0196; DRHO -0.1663; DTC -0.0544; GR 0.2431; NEUT 0.0846; PEF 0.0337; RESD -0.1794; RESM -0.2062; RESS -0.3069; SP 0.0932; DTS -0.0027; GR_CORR -0.0156; NEUT_CORR 0.1552; RESD_CORR -0.1411; RESS_CORR -0.0901; TEMP -0.0340; TENS -0.0449; 

In [22]:
preds = []
for i in range(len(df)):
    print(i)
    row = df.iloc[i]
    p = learn.predict(row)
#     print(p[0])
    preds.append(str(p[0]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


KeyboardInterrupt: 

In [17]:
(str(preds[0]))

'Matemateaonga Fm'

KeyError: 'tvdss'

In [93]:
preds = learn.pred_batch(ds_type=DatasetType.Test)

AttributeError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 595, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 104, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/tabular/data.py", line 125, in get
    codes = [] if self.codes is None else self.codes[o]
AttributeError: 'TabularList' object has no attribute 'codes'


In [ ]:
preds.size()

In [123]:
pandas.np.argmax(preds, axis=1)

tensor([20, 14, 11, 12, 11, 20, 11, 20,  4, 11, 14, 14,  2, 10, 11, 11, 11, 11,
        20, 11,  4, 19, 11, 11, 13, 20, 11, 14, 13,  4, 20, 11, 19, 11, 19, 13,
        14, 13,  2,  4, 18, 14, 11, 19,  4, 12, 11, 20,  2, 11, 20, 11,  4,  7,
        11, 14,  9, 14, 14, 11, 11, 20,  2, 14])

In [78]:
for i in df

well,BS_na,CALI_na,DENS_na,DRHO_na,DTC_na,GR_na,NEUT_na,PEF_na,RESD_na,RESM_na,RESS_na,SP_na,DTS_na,GR_CORR_na,NEUT_CORR_na,RESD_CORR_na,RESS_CORR_na,TEMP_na,TENS_na,tvdss,BS,CALI,DENS,DRHO,DTC,GR,NEUT,PEF,RESD,RESM,RESS,SP,DTS,GR_CORR,NEUT_CORR,RESD_CORR,RESS_CORR,TEMP,TENS,target,prediction
cheal-c4,False,False,False,True,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,-0.3085,-0.3150,-0.3156,-0.6459,-0.1189,0.5300,0.2566,-0.0955,-1.5681,0.0091,-0.0135,-0.0278,0.0816,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Matemateaonga Fm,Manganui Fm
cheal-g3,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.1384,-0.3150,-0.3121,-0.7046,0.2028,0.1768,0.2344,0.8437,-0.8302,-0.0061,0.0018,0.0112,-1.2173,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Matemateaonga Fm,Manganui Fm
cheal-a3x,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,False,False,1.1397,-1.6509,-1.6437,1.2682,2.1795,-0.7853,0.3637,-0.2680,1.5280,-0.0587,-0.0126,-0.0230,0.6207,-0.0042,-0.0175,0.4546,-0.1485,-0.0958,1.5204,0.9532,Mt Messenger,Matemateaonga Fm
cheal-g2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,-0.5796,-0.3150,0.0820,-1.1939,1.1355,1.1147,0.5826,1.3855,-0.6556,0.0374,0.0361,0.0284,0.0450,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Tangahoe Fm,Intra Kiore SST
cheal-g2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.6578,-0.3150,-0.3451,0.0978,1.9718,-0.2845,0.4115,0.0490,0.5364,-0.0482,-0.0377,-0.0225,-0.3710,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Urenui Fm,Tangahoe Fm
cheal-a10,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.9436,-0.3150,-0.0866,0.7437,-0.9873,-0.5304,1.1381,1.4397,1.1040,-0.0530,-0.0424,-0.0271,-0.4160,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Urenui A SST,Cheal SST
cheal-b1,False,False,False,False,False,False,False,True,False,True,False,False,True,True,False,True,True,False,False,-0.2338,-0.3150,-0.3795,-0.5353,-1.5032,0.6520,-0.0260,0.2303,0.0474,0.0125,-0.0126,0.0232,0.4185,-0.0042,-0.0175,1.1955,-0.1485,-0.0958,-0.7644,-0.5516,Matemateaonga Fm,Intra Kiore SST
cheal-b4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,-0.2970,-0.3150,-0.0479,-0.3005,-1.4624,0.5957,0.1839,0.9058,0.4600,-0.0116,-0.0146,-0.0169,-0.3687,-0.0042,-0.0175,2.3964,-0.1485,-0.0958,-0.0098,0.8399,Matemateaonga Fm,Manganui Fm
cheal-a7,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,False,False,0.7542,-0.3150,0.1105,0.4139,-0.3250,-0.5757,1.5481,-0.1674,0.9743,-0.0517,-0.0126,-0.0234,-4.1355,-0.0042,-0.0175,0.5039,-0.1485,-0.0958,0.9942,2.0522,Urenui Fm,Mt Messenger
cheal-a10,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,0.9766,-0.3150,-0.2102,0.9198,-1.3090,-0.4557,1.4561,1.3133,0.6325,-0.0534,-0.0426,-0.0275,-0.4245,-0.0042,-0.0175,0.1618,-0.1485,-0.0958,-0.0476,-0.0492,Urenui A SST,Cheal SST


NoneType

In [55]:
%time
for remove_well in df['well'].unique():
    holdout_data =  df[df['well'] == remove_well]
    train_data = df[df['well'] != remove_well]
    data = (TabularList.from_df(train_data, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                            .random_split_by_pct()
                           .label_from_df(cols=dep_var)
#                            .add_test(test)
                           .databunch())
    learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])
    print(f'Well removed: {remove_well}')
    learn.fit(3, 1e-2)
    learn.save(remove_well)
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs
Well removed: cheal-a10


epoch,train_loss,valid_loss,accuracy
1,0.259419,0.231429,0.921432
2,0.254127,0.298935,0.884145
3,0.237063,0.229286,0.921175


Well removed: cheal-a11


epoch,train_loss,valid_loss,accuracy
1,0.306744,0.229166,0.917172
2,0.258824,0.240925,0.910455
3,0.257394,0.206609,0.927672


Well removed: cheal-a12


...

epoch,train_loss,valid_loss,accuracy
1,0.294014,0.234002,0.900823
2,0.249048,0.196776,0.925948
3,0.256670,0.219367,0.917740


Well removed: cheal-b4


epoch,train_loss,valid_loss,accuracy
1,0.297294,0.288648,0.887257
2,0.262844,0.888087,0.741448
3,0.247727,0.200618,0.927469


Well removed: cheal-a7


epoch,train_loss,valid_loss,accuracy
1,0.296715,0.235736,0.909913
2,0.253036,0.405637,0.850164
3,0.245434,0.278187,0.892803


In [61]:
df[df.well=='cheal-a6'].head()

,tvdss,BS,CALI,DENS,DRHO,DTC,GR,NEUT,PEF,RESD,...,SP,fm,well,DTS,GR_CORR,NEUT_CORR,RESD_CORR,RESS_CORR,TEMP,TENS
157295,131.9100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Matemateaonga Fm,cheal-a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157296,131.9100,8.5,NaN,NaN,NaN,131.5667,NaN,NaN,NaN,NaN,...,NaN,Matemateaonga Fm,cheal-a6,NaN,NaN,NaN,NaN,NaN,NaN,1031.5211
157297,132.0624,8.5,NaN,NaN,NaN,131.4222,NaN,NaN,NaN,NaN,...,NaN,Matemateaonga Fm,cheal-a6,NaN,NaN,NaN,NaN,NaN,NaN,1030.6642
157298,132.2148,8.5,NaN,NaN,NaN,131.2778,NaN,NaN,NaN,NaN,...,NaN,Matemateaonga Fm,cheal-a6,NaN,NaN,NaN,NaN,NaN,NaN,1029.8073
157299,132.3672,8.5,NaN,NaN,NaN,131.1418,NaN,NaN,NaN,NaN,...,NaN,Matemateaonga Fm,cheal-a6,NaN,NaN,NaN,NaN,NaN,NaN,1028.9586


In [62]:
%%time
wells = df['well'].unique()
for [i,remove_well] in enumerate(wells):
    holdout_data =  df[df['well'] == remove_well]
    train_data = df[df['well'] != remove_well]
    data = (TabularList.from_df(train_data, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                            .random_split_by_pct()
                           .label_from_df(cols=dep_var)
#                            .add_test(test)
                           .databunch())
    learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])
    if i>0 : learn.load(wells[i-1])
    print(f'Well removed: {remove_well}')
    learn.fit(3, 1e-2)
    learn.save(remove_well)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs
Well removed: cheal-a10


epoch,train_loss,valid_loss,accuracy
1,0.285068,0.363313,0.864487
2,0.263447,0.300456,0.881752
3,0.245679,0.292121,0.927457


Well removed: cheal-a11


epoch,train_loss,valid_loss,accuracy
1,0.231014,0.167757,0.936629
2,0.202438,0.216949,0.914107
3,0.220571,0.379339,0.916998


Well removed: cheal-a12


epoch,train_loss,valid_loss,accuracy
1,0.216559,0.199881,0.916256
2,0.227283,0.370405,0.948541
3,0.209198,0.220496,0.933249


RuntimeError: Error(s) in loading state_dict for TabularModel:
	size mismatch for layers.6.weight: copying a param with shape torch.Size([21, 100]) from checkpoint, the shape in current model is torch.Size([18, 100]).
	size mismatch for layers.6.bias: copying a param with shape torch.Size([21]) from checkpoint, the shape in current model is torch.Size([18]).

In [10]:
for remove_well in data['well'].unique():
    holdout_data =  data[data['well'] == remove_well]
    train_data = data[data['well'] != remove_well]

    # Should still randomize the rows before passing to train
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    holdout_data = holdout_data.sample(frac=1).reset_index(drop=True)

    # Store target variable
    # Store categorical & numerical variable names
    target = 'fm'
    cat_names = ['well']
    num_names = data.columns.tolist()
    num_names.remove('well')
    num_names.remove('fm')
    
    train_data = train_data.dropna(subset=[target])
    holdout_data = holdout_data.dropna(subset=[target])
    
    X_train = train_data.drop(columns=target)
    X_test = holdout_data.drop(columns=target)
    y_train = train_data[target]
    y_test = holdout_data[target]
    
    scaler = StandardScaler()
    scaler.fit(X_train.drop(columns=cat_names))

    # Scale X_train's numerical variables
    transformed_X_train = scaler.transform(X_train.drop(columns=cat_names))
    transformed_X_train = pandas.np.concatenate((transformed_X_train, X_train[cat_names]), axis=1)

    # Scale X_test's numerical variables
    transformed_X_test = scaler.transform(X_test.drop(columns=cat_names))
    transformed_X_test = pandas.np.concatenate((transformed_X_test, X_test[cat_names]), axis=1)

    # Need to rebuild as dropping the columns prolly messed w/ the order
    feature_names = X_train.drop(columns=cat_names).columns.tolist() + cat_names

    # Putting them into df
    transformed_X_train = pandas.DataFrame(data=transformed_X_train, columns=feature_names)
    transformed_X_test = pandas.DataFrame(data=transformed_X_test, columns=feature_names)

    # Change to categorical type
    transformed_X_train[cat_names] = transformed_X_train[cat_names].astype('category')
    transformed_X_test[cat_names] = transformed_X_test[cat_names].astype('category')

    # No idea how this happened but somehow we lost the float datatype to object in the process
    transformed_X_train[num_names] = transformed_X_train[num_names].astype(float)
    transformed_X_test[num_names] = transformed_X_test[num_names].astype(float)

    # Labels was 1 to 9, but we need this to be 0 to 8
    oe = OrdinalEncoder()
    oe.fit(y_train.values.reshape(-1, 1))
    transformed_y_train = oe.transform(y_train.values.reshape(-1, 1))
    transformed_y_test = oe.transform(y_test.values.reshape(-1, 1))

    num_class = len(oe.categories_[0])

    train_data = lgb.Dataset(transformed_X_train, label=transformed_y_train.ravel())
    test_data = lgb.Dataset(transformed_X_test, label=transformed_y_test.ravel())

    params = {
        'objective':'multiclass',
        'num_class':num_class,
        'metric':'softmax',
    }

    model = lgb.train(params, train_set=train_data)

    # Output is the softmax probabilities so need to grab the class w/ highest confidence
    transformed_y_train_hat = pandas.np.argmax(model.predict(transformed_X_train), axis=1)
    transformed_y_test_hat = pandas.np.argmax(model.predict(transformed_X_test), axis=1)
    print(f'Well removed: {remove_well}')
    print(f1_score(transformed_y_test, transformed_y_test_hat, average='micro'))

Well removed: cheal-a10
0.38688930117501547
Well removed: cheal-a11
0.48366283398130533
Well removed: cheal-a12
0.5722606197095597


ValueError: Found unknown categories ['Tikorangi LST', 'MM1 SST', 'Lower Manganui Marker'] in column 0 during transform